Пары которые я взял в работу:
1) [x] Количество правонарушений зафиксированных полицией и количество бедных людей по регионам NUTS 
2) [x] Количество правонарушений зафиксированных полицией и численность населения по регионам NUTS
3) [x] Уровень безработицы по полу, возрасту и численность населения по регионам NUTS 
4) [x] Смертность и количество бедных людей по регионам NUTS
5) [x] Количество доступных коек в больницах и число лиц старше 65 лет по регионам NUTS

Корреляция, статистика по региону (топ 10)

# Сам код 

In [242]:
import pandas as pd
import numpy as np
from scipy.stats import t

# 1 пара

In [243]:
poor_people_data = pd.read_csv('/Users/vladimirbaranov/Downloads/count_of_poor_people.tsv', sep='\t')
crime_data = pd.read_csv('/Users/vladimirbaranov/Downloads/crime_data_by_country.tsv', sep='\t')

# Избавляемся от составных колонок
poor_people_data_cleaned = poor_people_data['freq,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)
crime_data_cleaned = crime_data['freq,unit,iccs,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Переименование колонок
poor_people_data_cleaned.columns = ['Frequency', 'Unit', 'Region']
crime_data_cleaned.columns = ['Frequency', 'Unit', 'ICCS_Code', 'Region']

poor_people_data_cleaned = pd.concat([poor_people_data_cleaned, poor_people_data.iloc[:, 1:]], axis=1)
crime_data_cleaned = pd.concat([crime_data_cleaned, crime_data.iloc[:, 1:]], axis=1)

# Замена отсутствующих значений
poor_people_data_cleaned.replace(':', pd.NA, inplace=True)
crime_data_cleaned.replace(':', pd.NA, inplace=True)

# Конвертация данных о годах в числовой формат
year_columns_poor = poor_people_data_cleaned.columns[3:]
year_columns_crime = crime_data_cleaned.columns[4:]

poor_people_data_cleaned[year_columns_poor] = poor_people_data_cleaned[year_columns_poor].apply(pd.to_numeric, errors='coerce')
crime_data_cleaned[year_columns_crime] = crime_data_cleaned[year_columns_crime].apply(pd.to_numeric, errors='coerce')

# Объединение данных по регионам
merged_data = pd.merge(poor_people_data_cleaned, crime_data_cleaned, on='Region', suffixes=('_poor', '_crime'))

# Определение валидных колонок по годам для расчёта корреляции
years_poor = [col for col in merged_data.columns if '_poor' in col]
years_crime = [col for col in merged_data.columns if '_crime' in col]

valid_years_poor = [col for col in years_poor if merged_data[col].notna().sum() > 0]
valid_years_crime = [col for col in years_crime if merged_data[col].notna().sum() > 0]

# Расчёт корреляций и p-value для каждого года
correlations = []

for year_poor, year_crime in zip(valid_years_poor[2:], valid_years_crime[2:]):
    year_data = merged_data[['Region', year_poor, year_crime]].dropna()
    if not year_data.empty:
        # Расчёт корреляции
        correlation = year_data[year_poor].corr(year_data[year_crime])

        # Расчёт значимости (p-value)
        n = len(year_data)  # Количество непустых наблюдений
        if n > 2 and not np.isnan(correlation):
            t_stat = correlation * np.sqrt((n - 2) / (1 - correlation**2))
            p_value = 2 * t.sf(abs(t_stat), df=n - 2)
        else:
            p_value = None

        correlations.append({'Year': year_poor.split('_')[0], 'Correlation': correlation, 'p-value': p_value})

# Преобразование в DataFrame для визуализации
correlation_df = pd.DataFrame(correlations)
correlation_df.sort_values(by='Year', inplace=True)

# Вывод результатов
correlation_df

,Year,Correlation,p-value
0,2014,-0.049249,0.606083
1,2015,-0.030797,0.174682
2,2016,-0.025449,0.272650
3,2017,-0.032950,0.113215
4,2018,-0.026560,0.162119
5,2019,-0.020548,0.276374
6,2020,-0.020880,0.269719
7,2021,-0.010204,0.557793
8,2022,-0.000308,0.984857


# Теперь сравнени по регионам

In [267]:
poor_people_data = pd.read_csv('/Users/vladimirbaranov/Downloads/count_of_poor_people.tsv', sep='\t')
crime_data = pd.read_csv('/Users/vladimirbaranov/Downloads/crime_data_by_country.tsv', sep='\t')

# Очистка названий колонок
poor_people_data.columns = poor_people_data.columns.str.strip()
crime_data.columns = crime_data.columns.str.strip()

# Разделение составных колонок для извлечения информации о регионах
poor_people_data_cleaned = poor_people_data['freq,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)
crime_data_cleaned = crime_data['freq,unit,iccs,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Переименование колонок для ясности
poor_people_data_cleaned.columns = ['Frequency', 'Unit', 'Region']
crime_data_cleaned.columns = ['Frequency', 'Unit', 'ICCS_Code', 'Region']

poor_people_data_cleaned = pd.concat([poor_people_data_cleaned, poor_people_data.iloc[:, 1:]], axis=1)
crime_data_cleaned = pd.concat([crime_data_cleaned, crime_data.iloc[:, 1:]], axis=1)

# Очистка названий колонок
poor_people_data_cleaned.columns = poor_people_data_cleaned.columns.str.strip()
crime_data_cleaned.columns = crime_data_cleaned.columns.str.strip()

# Отбор данных за 2014-2022 годы
years = [str(year) for year in range(2014, 2023)]
poor_people_data_cleaned = poor_people_data_cleaned[['Region'] + years].applymap(lambda x: str(x).strip())
crime_data_cleaned = crime_data_cleaned[['Region'] + years].applymap(lambda x: str(x).strip())

# Замена отсутствующих значений и фильтрация
poor_people_data_cleaned.replace(':', np.nan, inplace=True)
poor_people_data_cleaned = poor_people_data_cleaned[poor_people_data_cleaned.isna().sum(axis=1) < 2]

crime_data_cleaned.replace(':', np.nan, inplace=True)
crime_data_cleaned = crime_data_cleaned[crime_data_cleaned.isna().sum(axis=1) < 1]

# Объединение данных по регионам
merged_data = pd.merge(poor_people_data_cleaned, crime_data_cleaned, on='Region', suffixes=('_poor', '_crime'))

# Преобразование колонок в числовой формат
for year in years:
    merged_data[f'{year}_poor'] = pd.to_numeric(merged_data[f'{year}_poor'], errors='coerce')
    merged_data[f'{year}_crime'] = pd.to_numeric(merged_data[f'{year}_crime'], errors='coerce')

# Рассчёт корреляции и значимости для каждого региона
correlation_results = []

for region in merged_data['Region'].unique():
    region_data = merged_data[merged_data['Region'] == region]
    poor_series = region_data[[f'{year}_poor' for year in years]].values.flatten()
    crime_series = region_data[[f'{year}_crime' for year in years]].values.flatten()
    
    # Преобразуем в Series для корректной обработки
    poor_series = pd.Series(poor_series)
    crime_series = pd.Series(crime_series)
    
    # Рассчёт корреляции для временных рядов, игнорируя пропуски
    correlation = poor_series.corr(crime_series)
    
    # Рассчёт статистической значимости, если достаточно данных
    n = poor_series.count()  # Количество ненулевых точек
    if n > 2 and correlation is not np.nan:
        t_stat = correlation * ((n - 2) / (1 - correlation**2))**0.5
        p_value = 2 * t.sf(abs(t_stat), df=n-2)
    else:
        p_value = None  # Если данных недостаточно для расчёта значимости
    
    correlation_results.append({'Region': region, 'Correlation': correlation, 'p-value': p_value})

# Преобразование результатов в DataFrame для визуализации
correlation_df = pd.DataFrame(correlation_results)
correlation_df.sort_values(by='p-value')

/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/175982029.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  poor_people_data_cleaned = poor_people_data_cleaned[['Region'] + years].applymap(lambda x: str(x).strip())
/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/175982029.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crime_data_cleaned = crime_data_cleaned[['Region'] + years].applymap(lambda x: str(x).strip())


,Region,Correlation,p-value
110,NO02,0.306037,0.021799
75,ITC2,0.188106,0.066456
102,ME,0.183952,0.145652
64,ES64,-0.133214,0.161441
10,BG42,0.185029,0.172183
...,...,...,...
126,SE21,0.001508,0.988369
67,FI,0.001678,0.988836
69,FI1B,-0.001628,0.989168
68,FI1,0.000961,0.993607


# 2 пара

In [266]:
crime_data = pd.read_csv('/Users/vladimirbaranov/Downloads/crime_data_by_country.tsv', sep='\t')
population_data = pd.read_csv('/Users/vladimirbaranov/Downloads/population.tsv', sep='\t')

# Убираем пробелы в названиях колонок
crime_data.columns = crime_data.columns.str.strip()
population_data.columns = population_data.columns.str.strip()

# Разделение составных колонок на отдельные
crime_data[['freq', 'unit', 'iccs', 'geo']] = crime_data['freq,unit,iccs,geo\\TIME_PERIOD'].str.split(',', expand=True)
population_data[['freq', 'sex', 'unit', 'age', 'geo']] = population_data['freq,sex,unit,age,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Фильтрация данных по возрасту
population_data = population_data[population_data['age'] == 'TOTAL']

# Удаление оригинальных составных колонок
crime_data = crime_data.drop(columns=['freq,unit,iccs,geo\\TIME_PERIOD'])
population_data = population_data.drop(columns=['freq,sex,unit,age,geo\\TIME_PERIOD'])

# Определение годов для анализа (2014-2022)
common_years = {'2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'}
common_years_list = sorted(list(common_years))

# Фильтрация данных только по общим регионам
common_regions = set(crime_data['geo']).intersection(set(population_data['geo']))
crime_data_filtered = crime_data[crime_data['geo'].isin(common_regions)].applymap(lambda x: str(x).strip())
population_data_filtered = population_data[population_data['geo'].isin(common_regions)].applymap(lambda x: str(x).strip())

# Замена отсутствующих значений и фильтрация
crime_data_filtered.replace(':', np.nan, inplace=True)
crime_data_filtered = crime_data_filtered[crime_data_filtered.isna().sum(axis=1) < 1]

population_data_filtered.replace(':', np.nan, inplace=True)
population_data_filtered = population_data_filtered[population_data_filtered.isna().sum(axis=1) < 1]

# Приводим данные к числовому типу по нужным годам
for year in common_years_list:
    crime_data_filtered[year] = pd.to_numeric(crime_data_filtered[year], errors='coerce')
    population_data_filtered[year] = pd.to_numeric(population_data_filtered[year], errors='coerce')

# Суммируем данные по регионам для каждого года
crime_data_agg = crime_data_filtered.groupby('geo')[common_years_list].sum().reset_index()
population_data_agg = population_data_filtered.groupby('geo')[common_years_list].sum().reset_index()

# Объединение данных по регионам и годам
merged_data_regions = pd.merge(crime_data_agg, population_data_agg, on='geo', suffixes=('_crime', '_pop'))

# Вычисление корреляции и значимости для каждого региона
correlation_results = []

for region in merged_data_regions['geo'].unique():
    region_data = merged_data_regions[merged_data_regions['geo'] == region]
    crime_series = region_data[[f"{year}_crime" for year in common_years_list]].values.flatten()
    population_series = region_data[[f"{year}_pop" for year in common_years_list]].values.flatten()
    
    # Проверка, что временные ряды не пустые
    if not (pd.isnull(crime_series).all() or pd.isnull(population_series).all()):
        # Рассчёт корреляции, игнорируя NaN
        correlation = pd.Series(crime_series).corr(pd.Series(population_series))
        
        # Рассчёт p-значения, если достаточно данных
        n = pd.Series(crime_series).count()
        if n > 2 and not np.isnan(correlation):
            t_stat = correlation * np.sqrt((n - 2) / (1 - correlation**2))
            p_value = 2 * t.sf(abs(t_stat), df=n - 2)
        else:
            p_value = None
        
        correlation_results.append({'Region': region, 'Correlation': correlation, 'p-value': p_value})

# Преобразование результатов в DataFrame для наглядности
correlation_by_region_df = pd.DataFrame(correlation_results)
correlation_by_region_df.sort_values(by='Correlation', ascending=False)

/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/3916507565.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crime_data_filtered = crime_data[crime_data['geo'].isin(common_regions)].applymap(lambda x: str(x).strip())
/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/3916507565.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  population_data_filtered = population_data[population_data['geo'].isin(common_regions)].applymap(lambda x: str(x).strip())


,Region,Correlation,p-value
153,BG414,0.985196,0.000001
530,PT16E,0.984910,0.000001
144,BG341,0.984282,0.000002
128,BG312,0.982718,0.000002
126,BG31,0.980898,0.000003
...,...,...,...
553,SE11,-0.981479,0.000003
554,SE110,-0.981479,0.000003
97,BE3,-0.981806,0.000003
102,BE328,-0.984051,0.000002


# 3 пара

In [273]:
unemployment_df = pd.read_csv('/Users/vladimirbaranov/Downloads/estat_lfst_r_lfu2gac.tsv', sep='\t')
population_df = pd.read_csv('/Users/vladimirbaranov/Downloads/population.tsv', sep='\t')

# Очистка названий колонок
unemployment_df.columns = unemployment_df.columns.str.strip()
population_df.columns = population_df.columns.str.strip()

# Разделение колонок и выделение кода региона
unemployment_df[['freq', 'country_code', 'age', 'sex', 'unit', 'region_code']] = unemployment_df['freq,c_birth,age,sex,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)
population_df[['freq', 'sex', 'unit', 'age', 'region_code']] = population_df['freq,sex,unit,age,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Удаление исходных комбинированных колонок
unemployment_df.drop(columns=['freq,c_birth,age,sex,unit,geo\\TIME_PERIOD'], inplace=True)
population_df.drop(columns=['freq,sex,unit,age,geo\\TIME_PERIOD'], inplace=True)

# Отбор данных за 2014-2023 годы
years = [str(year) for year in range(2014, 2024)]
unemployment_filtered = unemployment_df[['region_code'] + years]
population_filtered = population_df[['region_code'] + years]

# Замена специальных символов на NaN
unemployment_filtered.replace({':': None, 'bu': None, 'u': None, 'b': None}, regex=True, inplace=True)

# Объединение данных по безработице и населению только для общих регионов
merged_data = pd.merge(unemployment_filtered, population_filtered, on='region_code', suffixes=('_безработица', '_население'))

# Преобразование колонок в числовой формат
for year in years:
    merged_data[f'{year}_безработица'] = pd.to_numeric(merged_data[f'{year}_безработица'], errors='coerce')
    merged_data[f'{year}_население'] = pd.to_numeric(merged_data[f'{year}_население'], errors='coerce')

# Рассчёт корреляции и значимости для каждого региона
correlation_results = []

for region in merged_data['region_code'].unique():
    region_data = merged_data[merged_data['region_code'] == region]
    unemployment_series = region_data[[f'{year}_безработица' for year in years]].values.flatten()
    population_series = region_data[[f'{year}_население' for year in years]].values.flatten()
    
    # Рассчёт корреляции для временных рядов
    correlation = pd.Series(unemployment_series).corr(pd.Series(population_series))
    
    # Рассчёт значимости (p-value), если достаточно данных
    n = pd.Series(unemployment_series).count()  # Количество ненулевых наблюдений
    if n > 2 and not np.isnan(correlation):
        t_stat = correlation * np.sqrt((n - 2) / (1 - correlation**2))
        p_value = 2 * t.sf(abs(t_stat), df=n - 2)
    else:
        p_value = None
    
    correlation_results.append({'Регион': region, 'Корреляция': correlation, 'p-value': p_value})

# Преобразование результатов в DataFrame для визуализации
correlation_df = pd.DataFrame(correlation_results)

# Настройки отображения для числовых значений
pd.options.display.float_format = '{:.6f}'.format

# Вывод отсортированного результата по корреляции
correlation_df[correlation_df['Корреляция'].notna()].sort_values(by='p-value')

/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/417475512.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment_filtered.replace({':': None, 'bu': None, 'u': None, 'b': None}, regex=True, inplace=True)
/Users/vladimirbaranov/PycharmProjects/pythonProject/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/vladimirbaranov/PycharmProjects/pythonProject/.venv/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,Регион,Корреляция,p-value
31,BG32,0.014421,0.118050
366,RO31,0.010589,0.206145
30,BG31,0.010271,0.242763
29,BG3,0.009194,0.251206
32,BG33,0.009689,0.272962
...,...,...,...
205,FRJ1,-0.000006,0.999286
444,UKD1,-0.000031,0.999451
0,AT,0.000001,0.999856
90,DE93,-0.000002,0.999868


# 4 пара

In [282]:
baby_deaths_df = pd.read_csv('/Users/vladimirbaranov/Downloads/rates_of_baby_deaths.tsv', sep='\t')
poverty_df = pd.read_csv('/Users/vladimirbaranov/Downloads/count_of_poor_people.tsv', sep='\t')

# Разделение составных колонок для извлечения информации о регионах
poverty_df_cleaned = poverty_df['freq,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)
baby_deaths_cleaned = baby_deaths_df['freq,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Переименование колонок
poverty_df_cleaned.columns = ['Frequency', 'Unit', 'Region']
baby_deaths_cleaned.columns = ['Frequency', 'Unit', 'Region']

poverty_df = pd.concat([poverty_df_cleaned, poverty_df.iloc[:, 1:]], axis=1)
baby_deaths_df = pd.concat([baby_deaths_cleaned, baby_deaths_df.iloc[:, 1:]], axis=1)

# Очистка названий колонок
poverty_df.columns = poverty_df.columns.str.strip()
baby_deaths_df.columns = baby_deaths_df.columns.str.strip()

# Отбор данных за 2014-2022 годы
years = [str(year) for year in range(2014, 2023)]
poverty_df = poverty_df[['Region'] + years].applymap(lambda x: str(x).strip())
baby_deaths_df = baby_deaths_df[['Region'] + years].applymap(lambda x: str(x).strip())

# Замена специальных символов на NaN и удаление строк с избыточными NaN
poverty_df.replace(':', np.nan, inplace=True)
poverty_df = poverty_df[poverty_df.isna().sum(axis=1) < 2]

baby_deaths_df.replace(':', np.nan, inplace=True)
baby_deaths_df = baby_deaths_df[baby_deaths_df.isna().sum(axis=1) < 1]

# Объединение данных по младенческой смертности и бедности только для общих регионов
merged_data = pd.merge(baby_deaths_df, poverty_df, on='Region', suffixes=('_deaths', '_poverty'))

# Преобразование колонок в числовой формат
for year in years:
    merged_data[f'{year}_deaths'] = pd.to_numeric(merged_data[f'{year}_deaths'], errors='coerce')
    merged_data[f'{year}_poverty'] = pd.to_numeric(merged_data[f'{year}_poverty'], errors='coerce')

# Рассчёт корреляции и значимости для каждого региона
correlation_results = []

for region in merged_data['Region'].unique():
    region_data = merged_data[merged_data['Region'] == region]
    deaths_series = region_data[[f'{year}_deaths' for year in years]].values.flatten()
    poverty_series = region_data[[f'{year}_poverty' for year in years]].values.flatten()
    
    # Рассчёт корреляции для временных рядов
    correlation = pd.Series(deaths_series).corr(pd.Series(poverty_series))
    
    # Рассчёт p-значения, если достаточно данных
    n = pd.Series(deaths_series).count()  # Количество ненулевых наблюдений
    if n > 2 and not np.isnan(correlation):
        t_stat = correlation * np.sqrt((n - 2) / (1 - correlation**2))
        p_value = 2 * t.sf(abs(t_stat), df=n - 2)
    else:
        p_value = None
    
    correlation_results.append({'Регион': region, 'Корреляция': correlation, 'p-value': p_value})

# Преобразование результатов в DataFrame для визуализации
correlation_df = pd.DataFrame(correlation_results)
correlation_df.sort_values(by='p-value')

/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/2892849159.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  poverty_df = poverty_df[['Region'] + years].applymap(lambda x: str(x).strip())
/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/2892849159.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  baby_deaths_df = baby_deaths_df[['Region'] + years].applymap(lambda x: str(x).strip())


,Регион,Корреляция,p-value
116,LU,0.943726,0.000132
133,RO,0.940691,0.000158
140,RO3,0.929008,0.000293
154,SE23,0.925940,0.000338
110,ITI,0.914624,0.000550
...,...,...,...
46,ES13,0.037529,0.923635
149,SE11,-0.029065,0.940831
151,SE2,-0.016438,0.966519
25,CZ05,0.014868,0.969716


# 5 пара

In [296]:
beds_df = pd.read_csv('/Users/vladimirbaranov/Downloads/available_beds.tsv', sep='\t')
population_data = pd.read_csv('/Users/vladimirbaranov/Downloads/population5.tsv', sep='\t')

# Разделение составных колонок для извлечения информации о регионах
beds_df_cleaned = beds_df['freq,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)
population_data_cleaned = population_data['freq,unit,sex,age,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Переименование колонок
beds_df_cleaned.columns = ['Frequency', 'Unit', 'Region']
population_data_cleaned.columns = ['Frequency', 'Unit', 'Sex', 'Age', 'Region']

beds_df = pd.concat([beds_df_cleaned, beds_df.iloc[:, 1:]], axis=1)
population_data = pd.concat([population_data_cleaned, population_data.iloc[:, 1:]], axis=1)
population_data = population_data.loc[population_data['Age'].isin(['Y65-69', 'Y70-74', 'Y75-79', 'Y80-84'])]

# Очистка названий колонок
beds_df.columns = beds_df.columns.str.strip()
population_data.columns = population_data.columns.str.strip()

# Отбор данных за 1993-2022 годы
years = [str(year) for year in range(1993, 2023)]
beds_df = beds_df[['Region'] + years].applymap(lambda x: str(x).strip())
population_data = population_data[['Region'] + years].applymap(lambda x: str(x).strip())

# Замена специальных символов на NaN и удаление строк с избыточными NaN
beds_df.replace(':', np.nan, inplace=True)
beds_df = beds_df[beds_df.isna().sum(axis=1) < 1]

population_data.replace(':', np.nan, inplace=True)
population_data = population_data[population_data.isna().sum(axis=1) < 1]

# Объединение данных по населению и койко-местам только для общих регионов
merged_data = pd.merge(population_data, beds_df, on='Region', suffixes=('_population', '_beds'))

# Преобразование колонок в числовой формат
for year in years:
    merged_data[f'{year}_population'] = pd.to_numeric(merged_data[f'{year}_population'], errors='coerce')
    merged_data[f'{year}_beds'] = pd.to_numeric(merged_data[f'{year}_beds'], errors='coerce')

merged_data.dropna(inplace=True)

# Рассчёт корреляции и значимости для каждого региона
correlation_results = []

for region in merged_data['Region'].unique():
    region_data = merged_data[merged_data['Region'] == region]
    population_series = region_data[[f'{year}_population' for year in years]].values.flatten()
    beds_series = region_data[[f'{year}_beds' for year in years]].values.flatten()
    
    # Рассчёт корреляции для временных рядов
    correlation = pd.Series(population_series).corr(pd.Series(beds_series))
    
    # Рассчёт p-значения, если достаточно данных
    n = pd.Series(population_series).count()  # Количество ненулевых наблюдений
    if n > 2 and not np.isnan(correlation):
        t_stat = correlation * np.sqrt((n - 2) / (1 - correlation**2))
        p_value = 2 * t.sf(abs(t_stat), df=n - 2)
    else:
        p_value = None
    
    correlation_results.append({'Регион': region, 'Корреляция': correlation, 'p-value': p_value})

# Преобразование результатов в DataFrame для визуализации
correlation_df = pd.DataFrame(correlation_results)
correlation_df.sort_values(by='p-value', ascending=False)

/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/1120710189.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  beds_df = beds_df[['Region'] + years].applymap(lambda x: str(x).strip())
/var/folders/74/r7prtqcj7dq_5mmv1b4tw2qm0000gn/T/ipykernel_5614/1120710189.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  population_data = population_data[['Region'] + years].applymap(lambda x: str(x).strip())


,Регион,Корреляция,p-value
0,AT,-0.002537,0.933635
12,BG32,-0.009076,0.881986
10,BG,-0.009255,0.761281
1,AT11,0.018502,0.726438
18,SI,-0.019014,0.532502
3,AT13,0.033444,0.527045
17,LT,-0.026488,0.384510
14,BG34,-0.071733,0.240098
15,BG41,-0.076244,0.211733
16,BG42,-0.082586,0.176040
